# Simulador de Lanzamiento de Dados

In [2]:
import random as rd
import matplotlib.pyplot as plt
import numpy as np

import sys
sys.path.append("../src")

from simulador import Simulador


In [3]:
caras_dado=6
nLanzamientos=1000000
nDados=2

In [4]:
Simulador.dados(caras_dado,nDados)
lanzamintos=Simulador.lanzamientos(nLanzamientos)
print(lanzamintos)

[[6 2]
 [2 5]
 [6 6]
 ...
 [4 6]
 [2 5]
 [2 3]]


In [5]:
def dato(ndado,caras,resultados):
    frecuencias={}
    for cara,resultado in zip(caras,resultados):
        frecuencias[int(cara)]=int(resultado)
    return {f"dado_{ndado}":frecuencias}

In [6]:
def frecuencia_absoluta(nDados,lanzamintos):
    resultados={}
    for i in range(nDados):
        cara_dado, frecuencias_cara= np.unique(lanzamintos[:, i], return_counts=True)
        resultados[f"dado_{i+1}"]=frecuencias_cara
    return resultados

In [7]:
# frecuencia absoluta
frecuencia=frecuencia_absoluta(nDados,lanzamintos)
frecuencia

{'dado_1': array([166616, 166753, 166130, 166185, 167732, 166584]),
 'dado_2': array([166754, 165978, 166530, 166605, 166984, 167149])}

In [8]:
# frecuencia relativa
def frecuencia_relativa(nLanzamientos,frecuencia):
    resultados={}
    for dado, frecuencias in frecuencia.items():
        resultados[dado]=frecuencias/nLanzamientos
    return resultados

frecuencia_relativa(nLanzamientos,frecuencia)

{'dado_1': array([0.166616, 0.166753, 0.16613 , 0.166185, 0.167732, 0.166584]),
 'dado_2': array([0.166754, 0.165978, 0.16653 , 0.166605, 0.166984, 0.167149])}

In [ ]:
import itertools
from collections import Counter

def probabilidad_teorica(caras_dado=6, nDados=1):
    combinaciones = itertools.product(range(1, caras_dado + 1), repeat=nDados)
    sumas = (sum(comb) for comb in combinaciones)
    conteo = Counter(sumas)
    total_combinaciones = caras_dado ** nDados
    
    probabilidad = {suma: conteo[suma] / total_combinaciones for suma in sorted(conteo)}

    return probabilidad

# Ejemplo
prob = probabilidad_teorica(6, 2)
for suma, p in prob.items():
    print(f"Suma {suma}: {p:.4f}")

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
Suma 2: 0.0278
Suma 3: 0.0556
Suma 4: 0.0833
Suma 5: 0.1111
Suma 6: 0.1389
Suma 7: 0.1667
Suma 8: 0.1389
Suma 9: 0.1111
Suma 10: 0.0833
Suma 11: 0.0556
Suma 12: 0.0278
